In [3]:
pip install ttkbootstrap


   ---------------------------------------- 0.0/133.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/133.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/133.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/133.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/133.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/133.0 kB ? eta -:--:--
   ------ --------------------------------- 20.5/133.0 kB 43.8 kB/s eta

In [ ]:
import ttkbootstrap as ttk
from ttkbootstrap.constants import *
import tkinter as tk
from tkinter import scrolledtext, messagebox
import openai
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# 使用您的API密钥和API URL
api_key = "sk-PobYySGnY9BoWle6hUbseBp5EXwnWAhIhGt0CNO9bCi0z7tP"
api_url = "https://api.moonshot.cn/v1"  # Moonshot API URL

# 设置API密钥和基础URL
openai.api_key = api_key
openai.api_base = api_url

# 初始化历史对话记录
history = [
    {"role": "system", "content": "你是由 Moonshot 提供的人工智能助手，你擅长中文和英文的对话。你会为用户提供安全、有帮助、准确的回答。"}
]

# 调用API函数
def call_kimi_api(query):
    history.append({
        "role": "user",
        "content": query
    })
    try:
        completion = openai.ChatCompletion.create(
            model="moonshot-v1-8k",  # 使用 Moonshot 模型
            messages=history,
            temperature=0.3,
        )
        result = completion.choices[0].message['content']
        # 更新历史对话
        history.append({
            "role": "assistant",
            "content": result
        })
        return result
    except Exception as e:
        return f"API 调用出错：{str(e)}"

# 创建一个Tkinter GUI
def create_gui():
    # 使用 ttkbootstrap 主题
    root = ttk.Window(themename="darkly")  # 你可以尝试其他主题如 "superhero", "flatly", "cosmo" 等
    root.title("Kimi API 助手")
    root.geometry("1200x800")  # 设置窗口大小

    # 左侧框架 (包含对话历史和代码输入)
    left_frame = ttk.Frame(root)
    left_frame.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)

    # 右侧框架 (包含生成的图像)
    right_frame = ttk.Frame(root)
    right_frame.grid(row=0, column=1, sticky="nsew", padx=10, pady=10)

    # 设置行和列的比例，使布局更均衡
    root.grid_columnconfigure(0, weight=1)
    root.grid_columnconfigure(1, weight=1)
    root.grid_rowconfigure(0, weight=1)

    # 左侧 - 创建一个滚动文本框，用于显示对话历史
    history_display = scrolledtext.ScrolledText(left_frame, width=60, height=20)
    history_display.pack(pady=10)

    # 左侧 - 创建一个输入框，让用户输入查询
    query_label = ttk.Label(left_frame, text="请输入查询:", font=("Helvetica", 12))
    query_label.pack()

    query_entry = ttk.Entry(left_frame, width=80)
    query_entry.pack(pady=5)

    # 定义按钮点击后的操作
    def on_submit():
        user_query = query_entry.get()
        if user_query:
            # 显示用户查询到历史窗口
            history_display.insert(tk.END, f"用户: {user_query}\n")
            query_entry.delete(0, tk.END)
            # 调用API
            response = call_kimi_api(user_query)
            # 显示API响应结果
            history_display.insert(tk.END, f"助手: {response}\n")
            history_display.see(tk.END)

    # 左侧 - 创建提交按钮
    submit_button = ttk.Button(left_frame, text="提交查询", command=on_submit, bootstyle=SUCCESS)
    submit_button.pack(pady=5)

    # 左侧 - 代码解析与绘图部分
    code_label = ttk.Label(left_frame, text="请输入Kimi助手返回的代码 (Python代码将直接执行并生成图像):", font=("Helvetica", 12))
    code_label.pack()

    code_input_textbox = scrolledtext.ScrolledText(left_frame, width=60, height=10)
    code_input_textbox.pack(pady=10)

    # 右侧 - 创建一个绘图区域
    plot_frame = ttk.Frame(right_frame)
    plot_frame.pack(fill="both", expand=True)

    # 定义解析代码并生成图像的操作
    def on_draw():
        input_code = code_input_textbox.get("1.0", tk.END).strip()
        if input_code:
            try:
                # 清除之前的图像
                for widget in plot_frame.winfo_children():
                    widget.destroy()

                # 创建新的图像并执行代码
                fig, ax = plt.subplots()  # 在此创建图像

                # 使用exec动态执行用户代码
                local_env = {'plt': plt, 'fig': fig, 'ax': ax}  # 提供给exec的上下文环境
                exec(input_code, local_env)  # 动态执行用户输入的代码

                # 在Tkinter中显示图像
                canvas = FigureCanvasTkAgg(fig, master=plot_frame)
                canvas.draw()
                canvas.get_tk_widget().pack(fill="both", expand=True)

            except Exception as e:
                messagebox.showerror("执行错误", f"代码执行出错：{str(e)}")
        else:
            messagebox.showwarning("输入错误", "请输入有效的代码。")

    # 左侧 - 创建代码解析提交按钮
    draw_button = ttk.Button(left_frame, text="执行代码并生成图像", command=on_draw, bootstyle=PRIMARY)
    draw_button.pack(pady=5)

    # 启动GUI主循环
    root.mainloop()

# 运行GUI
create_gui()
